<a href="https://colab.research.google.com/github/StankoDiego/SOA_EA_2/blob/main/HPC/Stanko_Diego_ejercicio1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Introducción

# 2. Armado del ambiente

## 2.1 Armado del ambiente en CPU

No se requiere ejecuciones previas para la ejecución del programa

##2.2 Armado del ambiente en GPU

Se requiere la instalación del modulo de Cuda para Python

In [10]:
!pip install pycuda

#3. Desarrollo

##3.1 Desarrollo en CPU

In [ ]:
#---------------------PARAMETROS-------------------------------
#@title 3.1.1 Parametros de ejecución {vertical-output: true}

cantidadElementos = 1000000#@param {type: "number"}

import numpy
import time

tiempoTotal = time.time() * 1000

vector = numpy.random.randint(1000, size = cantidadElementos)

longitud = len(vector)

print("Original:")
#print(vector)

#----------------------ALGORITMO----------------------------
tiempoBucle = time.time() *1000

n = len(vector) 
gap = n//2

while gap > 0: 

    for i in range(gap, n): 
        temp = vector[i] 
        j = i 
        while  j >= gap and vector[j-gap] >temp: 
            vector[j] = vector[j-gap] 
            j -= gap 

        vector[j] = temp 
    gap //= 2

#----------------------FINALIZACION----------------------------
tiempoBucle = (time.time() *1000) - tiempoBucle
print("Ordenado:")
#print(vector)

tiempoTotal = (time.time() *1000) - tiempoTotal
print("\n")
print("Tiempo del bucle: " , tiempoBucle , "[ms.]")
print("Tiempo del total: " , tiempoTotal , "[ms.]")

Original:
Ordenado:


Tiempo del bucle:  31284.083984375 [ms.]
Tiempo del total:  31298.551025390625 [ms.]


##3.2 Desarrollo en GPU

In [9]:
# --------------------------------------------
#@title 3.1 Parámetros de ejecución { vertical-output: true }

orden_polinomio =  10#@param {type: "number"}
orden_taylor = 3#@param {type: "number"}
punto_evalacion =  5#@param {type: "number"}
# --------------------------------------------

import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

import time
import numpy

tiempoTotal = (time.time() * 1000)

# vectores en CPU.
coeficientes_cpu = numpy.random.randint(1000, size = orden_polinomio + 1 )
coeficientes_cpu = coeficientes_cpu.astype(numpy.int32())

terminos_taylor_cpu = numpy.random.randint(1000, size = orden_taylor + 1 )
terminos_taylor_cpu = coeficientes_cpu.astype(numpy.float32())

resultado_cpu = numpy.empty_like(terminos_taylor_cpu)

# vectores en GPU.
coeficientes_gpu = cuda.mem_alloc(coeficientes_cpu.nbytes)
terminos_taylor_gpu = cuda.mem_alloc(terminos_taylor_gpu)

# CPU - Defino la función kernel que ejecutará en GPU.
module = SourceModule("""
__global__ void kernel_taylor( int n, float alfa, float *X, float *Y )
{
  int idx = threadIdx.x + blockIdx.x*blockDim.x;
  if( idx < n )
  {
    Y[idx]  = alfa*X[idx] + Y[idx];
  }
}
""") 
# CPU - Genero la función kernel.
kernel = module.get_function("kernel_taylor")

tiempo_gpu = (time.time() * 1000)

# GPU - Ejecuta el kernel.
# TODO: Falta consultar limites del GPU, para armar las dimensiones correctamente.
dim_hilo = 256
dim_bloque = numpy.int( (orden_polinomio + dim_hilo-1) / dim_hilo )
print( "Thread x: ", dim_hilo, ", Bloque x:", dim_bloque )

#TODO: Ojo, con los tipos de las variables en el kernel.
kernel( numpy.int32(orden_polinomio),numpy.float32(punto_evalacion), coeficientes_gpu, resultado_gpu, block=( dim_hilo, 1, 1 ),grid=(dim_bloque, 1,1) )

tiempo_gpu = (time.time() * 1000) - tiempo_gpu

"""
# CPU - Informo el resutlado.
print( "------------------------------------")
print( "X: " )
print( x_cpu )
print( "------------------------------------")
print( "Y: " )
print( y_cpu )
print( "------------------------------------")
print( "R: " )
print( r_cpu )
"""

tiempo_total = (time.time() * 1000) - tiempoTotal

print( "Orden de polinomio: ", orden_polinomio )
print( "Thread x: ", dim_hilo, ", Bloque x:", dim_bloque )
print("Tiempo CPU: ", tiempo_en_ms( tiempo_total ), "[ms]" )
print("Tiempo GPU: ", tiempo_en_ms( tiempo_gpu   ), "[ms]" )

LogicError: ignored

#4. Tabla de pasos

#5. Conclusiones